In [ ]:

import ipywidgets as widgets
from IPython.display import display, clear_output
import json
from datetime import datetime

todo_items = []

layout_full = widgets.Layout(width='100%')
layout_half = widgets.Layout(width='48%')
layout_third = widgets.Layout(width='32%')
layout_small = widgets.Layout(width='80px')
layout_button = widgets.Layout(width='48%', height='32px')

overall_progress_bar = widgets.FloatProgress(min=0, max=100, value=0, layout=layout_full)
overall_progress_label = widgets.Label(value='(0%)')

todo_list_box = widgets.VBox(layout=widgets.Layout(border='1px solid #ccc', padding='10px'))

def update_overall_progress():
    if not todo_items:
        overall_progress_bar.value = 0
        overall_progress_bar.style = {'bar_color': '#ffffff'}
        overall_progress_label.value = "(0%)"
        return
    avg = sum(i['progress'] for i in todo_items) / len(todo_items)
    overall_progress_bar.value = avg
    shade = int(255 - (avg * 2.55))
    hex_color = f'#{shade:02x}{shade:02x}{shade:02x}'
    overall_progress_bar.style = {'bar_color': hex_color}
    overall_progress_label.value = f"({int(avg)}%)"

def refresh_list():
    todo_list_box.children = []
    for idx, item in enumerate(todo_items, 1):
        progress_bar = widgets.FloatProgress(value=item['progress'], min=0, max=100, layout=layout_full)
        shade = int(255 - (item['progress'] * 2.55))
        hex_color = f'#{shade:02x}{shade:02x}{shade:02x}'
        progress_bar.style = {'bar_color': hex_color}
        slider = widgets.IntSlider(value=item['progress'], min=0, max=100, step=5, layout=layout_full)

        def on_progress_change(change, i=item, bar=progress_bar):
            i['progress'] = change['new']
            bar.value = change['new']
            shade = int(255 - (change['new'] * 2.55))
            hex_color = f'#{shade:02x}{shade:02x}{shade:02x}'
            bar.style = {'bar_color': hex_color}
            update_overall_progress()

        slider.observe(on_progress_change, names='value')

        complete_btn = widgets.Button(description='완료✅', layout=layout_button)
        delete_btn = widgets.Button(description='삭제❌', layout=layout_button)

        def on_complete_clicked(b, i=item):
            i['progress'] = 100
            update_overall_progress()
            refresh_list()

        def on_delete_clicked(b, i=item):
            todo_items.remove(i)
            update_overall_progress()
            refresh_list()

        complete_btn.on_click(on_complete_clicked)
        delete_btn.on_click(on_delete_clicked)

        box = widgets.VBox([
            widgets.HTML(f"<b># {idx} {item['title']}</b>"),
            widgets.HBox([widgets.Label("진행률", layout=layout_small), progress_bar]),
            slider,
            widgets.HBox([widgets.Label("마감일", layout=layout_small), widgets.Label(str(item['due']) if item['due'] else '-')]),
            widgets.HBox([widgets.Label("중요도", layout=layout_small), widgets.Label(item['priority'])]),
            widgets.HBox([complete_btn, delete_btn])
        ], layout=widgets.Layout(border='1px solid #aaa', padding='10px', margin='10px 0'))

        todo_list_box.children += (box,)

title_input = widgets.Text(placeholder='할 일 입력', layout=layout_full)
progress_input = widgets.IntSlider(value=0, min=0, max=100, step=5, layout=layout_full)
due_input = widgets.DatePicker(layout=layout_full)
priority_input = widgets.Dropdown(options=['낮음', '보통', '높음'], value='보통', layout=layout_full)
add_button = widgets.Button(description='추가', button_style='success', layout=layout_full)

def add_task(b):
    if title_input.value.strip():
        todo_items.append({
            'title': title_input.value.strip(),
            'progress': progress_input.value,
            'due': due_input.value,
            'priority': priority_input.value
        })
        title_input.value = ''
        progress_input.value = 0
        due_input.value = None
        priority_input.value = '보통'
        update_overall_progress()
        refresh_list()

add_button.on_click(add_task)

save_btn = widgets.Button(description='저장', button_style='info', layout=layout_half)
load_btn = widgets.Button(description='불러오기', button_style='warning', layout=layout_half)

def save_data(b):
    with open('todo_data.json', 'w') as f:
        json.dump([{**i, 'due': i['due'].strftime('%Y-%m-%d') if i['due'] else None} for i in todo_items], f)

def load_data(b):
    try:
        with open('todo_data.json', 'r') as f:
            raw = json.load(f)
            todo_items.clear()
            for i in raw:
                todo_items.append({
                    'title': i['title'],
                    'progress': i['progress'],
                    'due': datetime.strptime(i['due'], '%Y-%m-%d').date() if i['due'] else None,
                    'priority': i.get('priority', '보통')
                })
        update_overall_progress()
        refresh_list()
    except:
        print("불러오기 실패")

save_btn.on_click(save_data)
load_btn.on_click(load_data)

display(widgets.HTML("<h3>🗂️ DAVID mk26</h3>"))
display(widgets.VBox([
    widgets.HBox([widgets.Label("할 일", layout=layout_small), title_input]),
    widgets.HBox([widgets.Label("진행률", layout=layout_small), progress_input]),
    widgets.HBox([widgets.Label("마감일", layout=layout_small), due_input]),
    widgets.HBox([widgets.Label("중요도", layout=layout_small), priority_input]),
    add_button,
    widgets.HBox([save_btn, load_btn]),
    widgets.HBox([overall_progress_bar, overall_progress_label]),
    todo_list_box
]))

update_overall_progress()
refresh_list()
